<a href="https://colab.research.google.com/github/GaryPython/Cathay_LLM/blob/main/R2/R2_Langchain_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Introduction

LangChain 是圍繞 LLMs 構建的框架。我們可以將其用於聊天機器人、Question-Answering (QA)、摘要等等。

這個函式庫的核心思想是我們可以將不同的元件 “鏈結” 在一起，以創建更多元的 LLMs 應用。 Chain 來自幾個 Module 的多個組件：

1. **Prompt templates**：Prompt templates 是不同類型提示的範本。例如「 chatbot 」樣式模板、ELI5 問答等
2. **LLMs**：像 GPT-3、Mistral、Llama、Breeze、TAIDE 等大型語言模型
3. **Agents**：Agents 使用 LLMs 決定應採取的動作。可以使用網路搜尋(Google Search)或計算器(Python func)之類的工具，並將所有工具包裝成一個邏輯循環的操作。
4. **Memory**：短期記憶、長期記憶。

我們將從 Prompt templates 和 LLMs 的基礎知識開始。以下教學將提供兩個 LLMs 選項，包含 Hugging Face Hub 或 Hugging Face Pipeline 的模型。

- GitHub: https://github.com/langchain-ai/langchain
- Docs: https://python.langchain.com/en/latest/index.html

In [ ]:
! pip install --upgrade langchain
! pip install -q -U bitsandbytes
! pip install accelerate
! pip install langchain-community
! pip install torch --upgrade
! pip install transformers --upgrade
! pip install accelerate --upgrade
! pip install langchain-huggingface

In [ ]:
import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.schema import StrOutputParser
from langchain_huggingface import ChatHuggingFace

In [ ]:
from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

### 1. LLMs
透過 Langchain 載入 Huggingface 上的各種大型語言模型，在 Langchain 內模型可以分為

1. LLM 模式：給予文字輸入，然後文字輸出

2. Chat Models 模式：基於LLM模式的更進階的模式，他的輸入和輸出是格式化的chat messages

In [ ]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = """
提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: """
print(llm(prompt))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: 2022 年全球最賣座的電影是《黑亞當：黑寡婦傳奇》(Black Adam: Black Superhero Legend)。


In [ ]:
prompt = """
提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: """
print(llm(prompt))


提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: 目前尚未決定，需待 NBA 2023 賽季進行後結果才會出爐。


### 2. Prompt
一個好的 Prompt 通常包含以下四個組成部分：

1. **指示**: 告訴模型要做什麼，如何使用提供的信息，如何處理查詢，並建立輸出
2. **範例輸入**: 提供範例輸入，以向模型示範預期的內容
3. **範例輸出**: 提供對應的範例輸出
4. **查詢**: 您希望模型處理的實際輸入

以下介紹幾種在 LangChain 上使用 Pormpt 的方式

### 2.1 PromptTemplate

In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我一個笑話")
message = prompt_template.format()

# 模型生成
print(llm(message))

告訴我一個笑話

1. 在美國，如果你開車的速度超過了時速80公里，警察會追你。但是，如果你的車是紅色的，警察還會用閃光燈警告你減速。


In [ ]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我關於一個{content}的{adjective}笑話。")
message = prompt_template.format(adjective="悲傷的", content="數據科學家")

# 模型生成
print(llm(message))

告訴我關於一個數據科學家的悲傷的笑話。

在一個小村莊里，一位年輕的數據科學家被任命為村长。他非常努力地工作，希望能改善村民們的生活。有一天，他收到了一份報告，報告中提到：“如果我們在村子里的每个家庭都建一个水井，那么村民们可以更容易地獲得水源。”

數據科學家很感兴趣，于是决定去考察一下这个想法。他遍访了全村，了解了村民们对这一提案的看法。大部分人表示支持，但也有一些人提出了担忧。其中有两个老人家说道：“我们老一輩的人并没有什么问题，我们可以用现有的水源。你为什么要给新一代人带来麻煩呢？”

數據科学家想了想，觉得他们的观点很有道理。他决定先试着用现有的水源解决村民们的需求。他向政府请示，得到批准后，开始了项目。

数月后，数据科学家回到村庄，发现村民们的生活条件有所改变。水井已经建成，但很多人还是会出现缺水的情况。他问村民们原因，村民们回答说：“由于雨水少，河流干渴，所以水源不足。”

数据科学家又去考察了一下，确认了村民们的描述。他决定再找其他方法解決问题。他再次向政府请示，得到同意后，开始了新的计划。

数年过去了，村民们的生活条件依然没有太大改变。数据科学家感到很沮喪，他决定告诉村民们真相。他跟村民们讲述了他的故事，以及如何应对缺水等问题。

村民们听完之后，有些人表示理解，但有些人仍然不满。其中一名老年人说道：“为什么你一直在寻求外帮助，而不是尝试利用我们自己的資源？”

数据科学家深有感触，他决定重新思考问题。他回到村庄，与村民们一起研究如何利用自然资源，如雨雪、冰層和地下水。在一段时间内，他和村民们共同研制了一套有效的水利管理方案。

最后，村民们的生活条件有了很大改变。水井不再需要，河流也恢复了生命力。村民们感激不已，感謝数据科学家的努力。

从这个故事中，我们可以看到，数据科学家的努力并非白费。他终于找到了适合村庄的解決方案，使村民们的生活条件有所改变。


### 2.2 ChatPromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個擁有強大能力的 AI 機器人。你的名字是{name}。"),
        ("human", "你好，你好嗎？"),
        ("ai", "我很好，謝謝！"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(name="Bob", user_input="你叫什麼名字？")
print(chat_llm.invoke(messages).content)

### 2.3 混合使用

In [ ]:
system_prompt = "你是一個擁有強大能力的 AI 機器人。"
question_prompt = "告訴我關於一個{content}的{adjective}笑話。"

full_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_prompt),
    HumanMessage("你好，你好嗎？"),
    AIMessage("我很好，謝謝！"),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=question_prompt,
            input_variables=["content", "adjective"])
    )
])
full_prompt.pretty_print()

## 3. LLM Chain
對於簡單的任務，使用單一 LLM（大型語言模型）效果很好。然而，對於更複雜的任務，通常需要鍊式多個步驟和/或模型。

在LangChain中，可以使用傳統的 LLMChain，較新且建議的方法是 LangChain 表達式語言（LCEL）。

### 3.1 LLMChain

In [ ]:
template = """問題: {question}

Let's think step by step.

答案: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "NBA 2023 年總冠軍球隊是誰?"
print(llm_chain.invoke({"question": question})["text"])

### 3.2 LCEL（LangChain Expression Language）

In [ ]:
# 需要加入 Instruction 參數，才能告訴模型你的對話結束了，因目前使用的模型上不支援 chat 模式
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。",),
        ("human", "{question}"),
    ]
)

# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | StrOutputParser()

# 將問題透過參數化的方式帶入
print(llm_chain.invoke({"question": "機器學習和深度學習有什麼不同？"}))

```
# 此內容會顯示為程式碼
```

# 第二次作業

## Basline
請將提供的火災新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」，並以下方的 JSON 格式回傳，並轉成 Python Dict。

```json
{
    "起火點": "廚房",
    "起火原因": "電線起火",
    "起火時間": "早上9點",
    "死亡與否": false
}
```

提示：JSON Output Parser

In [ ]:
news = ["""
嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋昨深夜11點起火，地點位於台82線快速道路旁，濃煙竄天嚇壞許多民眾。警消獲報到場滅火，疑堆放大量塑膠製品迅速延燒，並傳出一名老人失聯，經徹夜搜索，消防人員今早9點多在旁民宅找到驚魂未定70歲男子，將他送醫。
據悉，水上鄉三界埔一處豬舍昨夜失火，現場疑堆放大量廢棄塑膠製品，導致火勢延燒迅速猛烈，鐵皮屋被燒到變形倒塌，火滅後仍持續悶燒，消防局出動20車、47人搶救，並出動大型機具開挖滅火，經徹夜9小時灌救滅火，初判燃燒面積約為2000平方公尺。
一名老翁平時住在廢棄豬舍附近，昨夜傳出疑似失聯，消防人員徹夜擴大搜索火場，並未在廢棄豬舍找到他，直到今天早上9點48分才在火場後方建築物2樓逃生梯發現，幸未被火勢延燒波及，只有建築物表面遭濃煙熏黑，老翁驚魂未定，但未受傷。
嘉義縣消防局表示，年約70歲男子待在後方建築物2樓逃生梯，意識清楚、無明顯外傷，給予救護處置後，由救護車預防性送往聖馬醫院診治。提醒民眾遇到火災切勿慌張，「小火快逃、濃煙關門」是火場逃生不二法門，且隨時注意逃生路徑，就能提高獲救機會。
""",
"""
台南市成功大學勝利校區體育館今天晚上9點多發生火災，消防局接獲報案派遣13車27人前往搶救，現場為2樓健身房的天花板管線冒煙，人員都已進行疏散，消防人員出水滅火，火勢約半小時後撲滅。
消防人員說，可能是體育館空調設備故障引起火災；台南市消防局說，起火真正原因需由消災調查科人員進一步鑑定查明。
台南市消防局救災救護指揮中心今晚9點32分接獲報案，成功大學勝利校區體育館發生火災，中心派遣東門等消防分隊13車27人前往搶救，消防人員到達現場回報為2樓健身房的天花板管線冒煙，已進行疏散。義消到達外觀有濃煙，校方有初期滅火。
據了解，2樓為健身房使用，無延燒之虞，館內人員皆撤出。消防人員出水降溫，及排煙。火勢於晚上10點3分撲滅。
""",
"""
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
"""]

## Advance
延續練習一，是否有辦法先判斷問題是否與火災的資訊擷取有關

有關則去擷取「起火點」、「起火原因」、「起火時間」、「是否有人死亡」

若無關則回答此問題無法回答

`注意`：不要使用一個 Pormpt 來解決問題，試著將透過 LCEL 串接兩個不同的 LLM Chain。

In [ ]:
news = [
"""
晶片大廠輝達（NVIDIA）執行長黃仁勳7日下午現身台北市內湖區基湖路上的輝達台灣分公司，據悉，此行他主要是為了與台灣區的駐派主管開會，而昨日才剛露面跟大家開心聚餐、喝到微醺的他，今日則重新以專業的態度及堅定的步伐，穩步快速的徑直進入公司，前去與其餘主管一起開會。
據悉，黃仁勳此行是為了參加「台北國際電腦展」才來台，而他6日晚間也特地在位於中山區的「欣葉台菜」與輝達內部的180名同仁，召開慶功宴，而黃仁勳自上個月底來台後，所到之處都也被粉絲包圍，媒體更是密切報導他的相關談話內容，而他昨晚現身街頭，也在台北市各地引起軒然大波。
據輝達台灣分公司官網顯示，團隊生產線從基礎發明到結合 GPU 的處理器，再到系統元件，乃至完全整合系統，同時將目標放在三大垂直市場：遊戲、專業視覺化與設計，以及高效能運算與巨量資料分析。針對每一垂直市場提供處理器、軟體、工具、行銷和專業知識的平台，另外更逐漸增加提供連線服務。
此外，團隊身為視覺運算的全球領導者，也銷售元件，並授權 IP 給希冀以豐富繪圖能力打造差異化裝置的各大 OEM，充分發揮自己這些市場所做的創新。而外傳黃仁勳即將於8日搭機離開台灣，也讓各界人士更加關注他的一舉一動。
""",
"""
嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋昨深夜11點起火，地點位於台82線快速道路旁，濃煙竄天嚇壞許多民眾。警消獲報到場滅火，疑堆放大量塑膠製品迅速延燒，並傳出一名老人失聯，經徹夜搜索，消防人員今早9點多在旁民宅找到驚魂未定70歲男子，將他送醫。
據悉，水上鄉三界埔一處豬舍昨夜失火，現場疑堆放大量廢棄塑膠製品，導致火勢延燒迅速猛烈，鐵皮屋被燒到變形倒塌，火滅後仍持續悶燒，消防局出動20車、47人搶救，並出動大型機具開挖滅火，經徹夜9小時灌救滅火，初判燃燒面積約為2000平方公尺。
一名老翁平時住在廢棄豬舍附近，昨夜傳出疑似失聯，消防人員徹夜擴大搜索火場，並未在廢棄豬舍找到他，直到今天早上9點48分才在火場後方建築物2樓逃生梯發現，幸未被火勢延燒波及，只有建築物表面遭濃煙熏黑，老翁驚魂未定，但未受傷。
嘉義縣消防局表示，年約70歲男子待在後方建築物2樓逃生梯，意識清楚、無明顯外傷，給予救護處置後，由救護車預防性送往聖馬醫院診治。提醒民眾遇到火災切勿慌張，「小火快逃、濃煙關門」是火場逃生不二法門，且隨時注意逃生路徑，就能提高獲救機會。
""",
"""
對貓咪來說，貓跳台就是一個可以休息、運動的空間，是許多貓奴家裡的必備物品。根據日本寵物網《ねこちゃんホンポ》專欄作家指出，貓跳台的擺放位置非常重要，如果放在下述４個NG地點，不僅可能傷害到貓咪，連人類也有受傷的風險，鏟屎官一定要留意。
1.冷氣直吹：很多飼主都會忽略冷氣的位置，把貓跳台放在冷氣出風口附近，冷氣、暖氣對著貓跳台直直吹，貓咪很容易感冒打噴嚏、流鼻涕，還可能引起腹瀉等健康問題。
2.靠近電暖器、電暖扇：無論貓咪的運動神經有多好，都還是會不小心沒踩穩從跳台上摔下來，如果墜落的地方有擺放電暖器、電暖扇等發熱型電器，貓咪可能會被燙傷，嚴重也可能引起火災。
3.靠近沒有防護的窗戶：很多飼主都會把貓跳台放在窗戶附近，讓貓咪曬太陽、看風景。但是如果貓跳台附近的窗戶會一直開關，貓咪就會有更多逃跑的機會，即使有裝紗窗，貓咪的重量也可能導致紗窗脫落，如果住在高樓層，貓咪就可能墜樓喪命。所以如果想將貓跳台放在窗戶附近，請務必做好窗戶防護。
4.床邊或嬰兒床附近：放置貓跳台時，不僅要確保貓咪的安全，也要確保與貓一起生活的人類的安全，因為貓跳台使用久了也有倒塌或斷裂的風險，如果放在床邊或嬰兒床附近，就可能會弄傷睡覺的主人跟寶寶。
""",
"""
台南市成功大學勝利校區體育館今天晚上9點多發生火災，消防局接獲報案派遣13車27人前往搶救，現場為2樓健身房的天花板管線冒煙，人員都已進行疏散，消防人員出水滅火，火勢約半小時後撲滅。
消防人員說，可能是體育館空調設備故障引起火災；台南市消防局說，起火真正原因需由消災調查科人員進一步鑑定查明。
台南市消防局救災救護指揮中心今晚9點32分接獲報案，成功大學勝利校區體育館發生火災，中心派遣東門等消防分隊13車27人前往搶救，消防人員到達現場回報為2樓健身房的天花板管線冒煙，已進行疏散。義消到達外觀有濃煙，校方有初期滅火。
據了解，2樓為健身房使用，無延燒之虞，館內人員皆撤出。消防人員出水降溫，及排煙。火勢於晚上10點3分撲滅。
""",
"""
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
""",
"""
桃園楊梅幼獅工業區一間工廠深夜發生大火，濃煙竄天，消防出動66人漏夜搶救，派出高效能化學消防車「美洲豹」馳援搶救，幸好沒有人員受傷，詳細起火原因，有待火調科後續釐清。
鐵皮工廠深夜時分冒出熊熊烈火，濃煙不斷竄出火光照亮天空，燃燒面積相當大，附近住戶飽受驚嚇，甚至有住戶在火災發生後，聽工廠內傳出猛烈爆炸聲。
5號晚上11點桃園楊梅獅一路，一間鐵皮工廠發生火警，警消獲報後派遣28輛消防車、66人到場，立刻佈水線搶救，同時派出高效能化學車「美洲豹」到場馳援。這間工廠位於幼獅工業區內，是塑膠產業公司，面積將近千坪，火勢蔓延非常快速，詳細起火原因，有待火調科後續釐清。
"""]